# Linear Regression Simplification

In [5]:
import torch
from torch import nn
from torch.nn import init
import torch.optim as optim
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import random

##  生成数据集
我们生成与上一节相同的数据集，其中`features`是训练数据特征，`labels`是标签。

In [6]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)


## 读取数据
PyTorch提供了`data`包来读取数据。由于`data`常用作变量名，我们将导入的`data`模块用`Data`代替。在每一次迭代中，我们将随机读取包含10个样本的小批量。

In [7]:
import torch.utils.data as Data
batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

这里的`data_iter`和上一节中的一样。让我们读取并打印小批量数据样本。

In [8]:
for X, y in data_iter:
    print(X, y)

tensor([[ 1.0562e+00,  4.8149e-01],
        [ 3.6190e-02, -1.2309e+00],
        [-2.4813e-01,  1.0104e+00],
        [-9.7184e-02, -1.5699e-01],
        [ 2.4868e+00,  3.3901e-02],
        [ 2.1493e-03,  1.2981e-01],
        [ 3.2738e-01,  1.0248e-02],
        [-9.7520e-01,  5.0003e-01],
        [ 3.5258e-02,  4.7666e-01],
        [-1.7466e+00, -7.5044e-01]]) tensor([4.6759, 8.4641, 0.2703, 4.5533, 9.0739, 3.7757, 4.8310, 0.5364, 2.6594,
        3.2815])
tensor([[ 1.0704, -0.6758],
        [ 1.4635,  0.3698],
        [-0.7128,  1.5996],
        [-0.6461,  0.0994],
        [ 0.3579,  1.5187],
        [-0.2977,  0.6956],
        [-0.2678, -0.9193],
        [ 0.2822,  1.5234],
        [ 1.8422,  1.2524],
        [-0.7927, -0.2870]]) tensor([ 8.6254,  5.8679, -2.6555,  2.5857, -0.2393,  1.2363,  6.7883, -0.4112,
         3.6297,  3.5711])
tensor([[ 0.3488, -0.8462],
        [ 0.8871, -0.8729],
        [-1.6214, -1.7935],
        [-1.3591, -0.1726],
        [ 0.2007, -0.7167],
        [-0.31

## 定义模型
PyTorch提供了大量预定义的层，这使我们只需关注使用哪些层来构造模型。下面将介绍如何使用PyTorch更简洁地定义线性回归。
首先导入`torch.nn`模块。之前我们使用了`autograd`这里`nn`就是使用了`autograd`来定义模型。`nn`的核心数据结构是`Module`，它是一个抽象概念，可以表示神经网络中的某个层，也可以表示一个包含很多层的神经网络。在实际使用中，最常见的做法是继承`nn.Module`，撰写自己的网络/层。一个`nn.Module`实例应该包含一些层以及返回输出的前向传播(forward)方法。下面先来看看如何用`nn.Module`实现一个线性回归模型。

In [16]:
import torch.nn
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    def forward(self, x):
        y = self.linear(x)
        return y
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


可以通过`net.parameters()`来查看模型所有的可学习参数，此函数将返回一个生成器。

In [17]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.3620,  0.1003]], requires_grad=True)
Parameter containing:
tensor([0.0771], requires_grad=True)


## 初始化模型参数
在使用`net`前，我们需要初始化模型参数，如线性回归模型中的权重和偏差。PyTorch在`init`模块中提供了多种参数初始化方法。通过`init.normal_`将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布。偏差初始化为0。 

In [18]:
init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)

Parameter containing:
tensor([0.], requires_grad=True)

## 定义损失函数
我们采用PyTorch的均方误差损失作为模型的损失函数。

In [19]:
loss = nn.MSELoss()

## 定义优化算法
我们无须自己实现小批量随机梯度下降算法。`torch.optim`模块提供了很多常用的优化算法比如SGD、Adam等。下面我们创建一个用于优化`net`所有参数的优化器实例。

In [25]:
optimizer = optim.SGD(net.parameters(), lr = 0.001)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


## 训练模型

In [26]:
num_epoch = 2000
for epoch in range(1, num_epoch + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1,1))
        optimizer.zero_grad() # 梯度清零
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000073
epoch 2, loss: 0.000061
epoch 3, loss: 0.000116
epoch 4, loss: 0.000048
epoch 5, loss: 0.000097
epoch 6, loss: 0.000132
epoch 7, loss: 0.000080
epoch 8, loss: 0.000076
epoch 9, loss: 0.000150
epoch 10, loss: 0.000183
epoch 11, loss: 0.000051
epoch 12, loss: 0.000088
epoch 13, loss: 0.000079
epoch 14, loss: 0.000239
epoch 15, loss: 0.000111
epoch 16, loss: 0.000051
epoch 17, loss: 0.000084
epoch 18, loss: 0.000116
epoch 19, loss: 0.000061
epoch 20, loss: 0.000138
epoch 21, loss: 0.000121
epoch 22, loss: 0.000087
epoch 23, loss: 0.000148
epoch 24, loss: 0.000076
epoch 25, loss: 0.000192
epoch 26, loss: 0.000148
epoch 27, loss: 0.000172
epoch 28, loss: 0.000078
epoch 29, loss: 0.000036
epoch 30, loss: 0.000098
epoch 31, loss: 0.000110
epoch 32, loss: 0.000062
epoch 33, loss: 0.000072
epoch 34, loss: 0.000128
epoch 35, loss: 0.000127
epoch 36, loss: 0.000106
epoch 37, loss: 0.000239
epoch 38, loss: 0.000050
epoch 39, loss: 0.000158
epoch 40, loss: 0.000224
epoch 41,

In [27]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 2.0000, -3.4000]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1999], requires_grad=True)
